<a href="https://colab.research.google.com/github/blue-create/langlens/blob/main/scripts/filter_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports and Paths

In [1]:
# imports
import os
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
from tqdm import tqdm 


In [2]:
# set paths
data_path="drive/MyDrive/Work/Frontline/data/"
unzipped_path = data_path+"unzipped"
json_path = data_path+"json/"


### Methods and Constants

In [4]:
# methods

def filter_strings(text, combination_list):
  """ function to compare if any of a list of words occur in a text or or list of texts 
  Parameters:
    - text (str or list of str): text which is checked 
    - combination_list (list of str): list of words which are checked if they occur in text 
  Returns:
    - boolean: True or False depending on if any of the words in combination_list occurs in text
  """

  text="".join(text)
  combinations = combination_list
  for comb in combinations:
    
    if all(word in text for word in comb.split(" und ")):
        return True
  return False

In [5]:
comb1 = ['häuslich gewalt',
        'partnerschaftsgewalt',
        'partnergewalt',
        'femizid',
        'beziehungstat',
        'liebesdrama',
        'ehedrama',
        'liebestragödie',
        'ehelich gewalt',
        'ehekrieg',
        'innerfamiliär gewalt',
        'innerhäuslich gewalt',
        'gewalt und \sehe',
        'ehe und hölle'
        'gewalt und (freund(in)?|partner(in)?|mann|frau)\s',
        'vergewaltigen und (freund(in)?|partner(in)?|mann|frau)\s',
        'vergewaltigung und (freund(in)?|partner(in)?|mann|frau)\s',
        'missbrauch und (freund(in)?|partner(in)?|mann|frau)\s',
        'missbräuchlich und (freund(in)?|partner(in)?|mann|frau)\s',
        'gewalttätig und (freund(in)?|partner(in)?|mann|frau)\s',
        'verletzung und (freund(in)?|partner(in)?|mann|frau)\s',
        'verletzen und (freund(in)?|partner(in)?|mann|frau)\s',
        'übergriffig und (freund(in)?|partner(in)?|mann|frau)\s',
        'drohung und (freund(in)?|partner(in)?|mann|frau)\s',
        'drohen und (freund(in)?|partner(in)?|mann|frau)\s',
        'manipulation und (freund(in)?|partner(in)?|mann|frau)\s',
        'manipulieren und (freund(in)?|partner(in)?|mann|frau)\s',
        'beleidigen und (freund(in)?|partner(in)?|mann|frau)\s',
        'beleidigung und (freund(in)?|partner(in)?|mann|frau)\s',
        'gaslighting und (freund(in)?|partner(in)?|mann|frau)\s',
        'schlagen und (freund(in)?|partner(in)?|mann|frau)\s',
        'zwingen und (freund(in)?|partner(in)?|mann|frau)\s', 
        'gezwungen und (freund(in)?|partner(in)?|mann|frau)\s',
        'zwang und (freund(in)?|partner(in)?|mann|frau)\s',
        'einsperren und (freund(in)?|partner(in)?|mann|frau)\s',
        'stalking und (freund(in)?|partner(in)?|mann|frau)\s',
        'stalken und (freund(in)?|partner(in)?|mann|frau)\s',
        'kontrollieren und (freund(in)?|partner(in)?|mann|frau)\s',
        'kontrolle und (freund(in)?|partner(in)?|mann|frau)\s',
        'isolieren und (freund(in)?|partner(in)?|mann|frau)\s',
        'isolation und (freund(in)?|partner(in)?|mann|frau)\s',
        'hauen und (freund(in)?|partner(in)?|mann|frau)\s',
        'ohrfeige und (freund(in)?|partner(in)?|mann|frau)\s'

        ]

### Filter based on key words

In [6]:
DV_art=[]

In [ ]:
for json in tqdm(os.listdir(json_path)):
  df_temp=pd.read_json(json_path+"/"+json)
  for row_j in tqdm(range(len(df_temp))):
    for art_i in range(len(df_temp.loc[row_j,:][0])):
      if filter_strings(df_temp.loc[row_j,:][0][art_i]["text"],comb1):
        DV_art.append(df_temp.loc[row_j,:][0][art_i])